The code below creates a randomized trial order that is counterbalanced and checked for order effects, using 
balanced Latin squared. The challenge is, based on the number of conditions and number of repetitions you want to have
in a run (neuroimaging, behavioural, anything!), it might be harder to check for order effects, or counterbalance
perfectly.



Balanced Latin squares counterbalances and basically eliminates order effects perfectly, but for this it needs to 
have the same amount of conditions as repetitions, which usually isn't the case. My code below basically creates
a balanced latin square for the smaller number, then adds what is leftover to the matrix by checking for order
effects in each addition-- well almost perfectly, I need to work more on that.


At the last portion, I wrote something which checks for order effects: for now, if you have some orders
appearing more than others, (haven't come across much with this pipeline), you can run this again:
wanted to do this iteratively but it took so much time! and really sometimes you don't need something perfect/
by mathematical constraints cannot have something perfect
                             

In [126]:
import random
import numpy as np 
import itertools
import pandas as pd
from itertools import permutations

In [127]:
Total_block = 4 #how many rep 
body_parts = ["feet", "hand", "torso", "arm", "chin"]
Big_block = []
conditions = len(body_parts) 
body_parts = sorted(body_parts, key=len) 

The original version of the code chunk below is from here: https://github.com/loutfouz/ballatsq/blob/master/ballatsq.py
Thank you!

In [128]:
def transposed(lists):
   if not lists: return []
   return map(lambda *row: list(row), *lists)
   
def balanced_sq(size):
    first_row = [1,2]
    j = size
    for i in range(size,2,-2):
        first_row.append(j)
        j-=1
    j = 3
    for i in range(3,size,2):
        first_row.insert(i,j)
        j+=1
	#initial values were generated, 
	#now let's fill these rows using circular shift
    ballatsq = []
    for i in range(0,size):
        ballatsq.append([first_row[i]])
        for j in range(0,size-1):
            next_val = 1+(ballatsq[i][j])%(size) 
            ballatsq[i].append(next_val)

    return transposed(ballatsq)

In [129]:
Big_block_sq = list(balanced_sq(Total_block))
added_condition_total = conditions if conditions-Total_block == 1  else np.arange(Total_block, conditions)+1
#if let's say you have 4 repetitions but 6 body parts, you would want to add the 5th and 6th body part to the 
#perfectly made Latin square 1 by 1, in our case we don't need this so didn't spend time to implement this, 
#just putting the markers for it for future structuring 


This chunk below inserts the remaining condition to the balanced latin square in a counterbalanced way: ex. 5 will be 
added once before 1, once before 2, once before 3, once before 4. Random sampling is done to randomize "which" 4, 
which 3 etc. 



TODO is looping this for added_condition_total:add a for loop: for added_condition in added_condition_total, in case let's say there are 4 repetitions but you have 
7 body parts

In [108]:

Big_block = []
counter = np.zeros((0, Total_block))
elements_to_pick = random.sample(range(Total_block+1)[1:], Total_block)
for i in range(Total_block):
    single_list = Big_block_sq[i]
    num = elements_to_pick[i]
    j = single_list.index(num)
    single_list.insert(j,added_condition)
    Big_block.append(single_list)

In [133]:
random.shuffle(body_parts)
mymap = {1:body_parts[0], 2:body_parts[1], 3:body_parts[2], 4:body_parts[3], 5:body_parts[4]} #future TODO is to define that 1:5 range based on conditions
Big_block_body= pd.DataFrame(Big_block).applymap(lambda s: mymap.get(s) if s in mymap else s)
#This way you can create many as body_parts! random orders but they will be counterbalanced/ordered

In [135]:
#This is to check for order effects, if a certain sequence appereance is more than any other. You can run this with
#pretty much anything, given you define Big_block_body and body_parts

Bigblock = Big_block_body.values.flatten()
comb_list = list(permutations(body_parts, 2))
combinations = pd.DataFrame(-1, columns=comb_list, index = np.arange(4) )
for i in range(len(Bigblock)-1):
    our_order = "('" + Bigblock[i] + "', '" + Bigblock[i+1] + "')" 
    for a in range(len(comb_list)):
        if our_order == '{}'.format(comb_list[a]):
            combinations.iloc[0,a] = combinations.iloc[0,a] + 1
            #i will also record the i : starting block this happens if i need to do switches
            combinations.iloc[np.where(combinations[comb_list[a]] == -1)[0][0],a] = i 
combinations.loc[0] = combinations.loc[0].add(1)  #just for easy reading, so combinations.loc[0] shows how many of that combination exists
combinations= combinations.transpose()

In [136]:
combinations
#0 corresponds to how many times you see that sequence, 1 2 3 are noting the starting indices in case you want to
#go back and change etc

#You can see how the one matrix I created is perfectly balanced for ordering effects

,0,1,2,3
"(hand, arm)",1,1,-1,-1
"(hand, feet)",1,16,-1,-1
"(hand, torso)",1,14,-1,-1
"(hand, chin)",1,7,-1,-1
"(arm, hand)",1,13,-1,-1
"(arm, feet)",1,5,-1,-1
"(arm, torso)",1,2,-1,-1
"(arm, chin)",0,-1,-1,-1
"(feet, hand)",1,6,-1,-1
"(feet, arm)",1,4,-1,-1
